# Workshop Notebook - July 16, 2025

Welcome to today's workshop! This notebook contains some basic examples to get started.

## Overview
- Basic Python operations
- Simple calculations
- String manipulation

In [8]:
def get_baml_client():
  """
  a bunch of fun jank to work around the google colab import cache
  """

  from google.colab import userdata
  import os
  os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

  import importlib
  import baml_client
  importlib.reload(baml_client)
  return baml_client.sync_client.b



In [1]:
!pip install baml-py


In [2]:
!baml-cli init


In [3]:
!baml-cli generate

2025-07-16T01:14:07.315 [BAML INFO] Wrote 13 files to baml_client
2025-07-16T01:14:07.315 [BAML INFO] Generated 1 baml_client: ../baml_client


In [6]:
!baml-cli test

Summary: 0/2 tests 
Summary: 0/2 tests 
Summary: 0/2 tests 
Summary: 0/2 tests 
⠁ Running DetermineNextStep::HelloWorld
Summary: 0/2 tests 
⠁ Running DetermineNextStep::HelloWorld
Summary: 0/2 tests 
⠁ Running DetermineNextStep::HelloWorld
⠁ Running ExtractResume::vaibhav_resume                                         2025-07-16T01:14:25.520 [BAML WARN] Function DetermineNextStep:
    Client: Qwen3 (<unknown>) - 0ms
    ---PROMPT---
    system: /nothink 
    
    You are a helpful assistant that can help with tasks.
    user: You are working on the following thread:
    
    {
      "type": "user_input",
      "data": "hello!"
    }
    
    What should the next step be?
    
    Answer in JSON using this schema:
    {
      intent: "done_for_now",
      message: string,
    }
    
    ---REQUEST OPTIONS---
    ---ERROR (Unspecified error code: 2)---
    Failed to build request: reqwest::Error {
        kind: Builder,
        source: RelativeUrlWithoutBase,
    }
Summary: 1/2 tests - 1

In [7]:
!rm baml_src/resume.baml

now lets download the agent

In [13]:
!curl -fsSL -o baml_src/agent.baml https://raw.githubusercontent.com/humanlayer/12-factor-agents/refs/heads/main/workshops/2025-07-16/walkthrough/01-agent.baml && cat baml_src/agent.baml

class DoneForNow {
  intent "done_for_now"
  message string 
}

function DetermineNextStep(
    thread: string 
) -> DoneForNow {
    client "openai/gpt-4o"

    // use /nothink for now because the thinking tokens (or streaming thereof) screw with baml (i think (no pun intended))
    prompt #"
        {{ _.role("system") }}

        You are a helpful assistant that can help with tasks.

        {{ _.role("user") }}

        You are working on the following thread:

        {{ thread }}

        What should the next step be?

        {{ ctx.output_format }}
    "#
}

test HelloWorld {
  functions [DetermineNextStep]
  args {
    thread #"
      {
        "type": "user_input",
        "data": "hello!"
      }
    "#
  }
}

In [14]:
!baml-cli generate

2025-07-16T01:22:21.660 [BAML INFO] Wrote 13 files to baml_client
2025-07-16T01:22:21.660 [BAML INFO] Generated 1 baml_client: ../baml_client


In [9]:
b = get_baml_client()

step = b.DetermineNextStep("hi")
print(step)

intent='done_for_now' message='Hello! Please let me know if there is anything specific you need assistance with.'
